In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import numpy as np 
import pandas as pd
import re
import warnings
import time

warnings.filterwarnings("ignore") # И пусть весь мир помолчит.

browser = webdriver.Safari()   # Решили, что selenium, так как запросы через requests отправляют нас на капчу.
browser.get('https://auto.ru/cars/used/?page=1') # fake_useragent тоже не помог.

data = pd.DataFrame({'Модель': [], 
                     'Цена': [], 
                     'Год выпуска': [],
                     'Пробег': [],
                     'Привод': [],
                     'ЛС': [],
                     'Тип двигателя': []
                     , 'Тип кузова': [],
                     'Коробка передач': [],
                     'Объем двигателя': [],
                     'Цвет' : []})

In [2]:
for i in np.arange(100):
    
    html = browser.page_source
    soup = BeautifulSoup(html)

    other = soup.find_all(attrs={"class":{"ListingItemTechSummaryDesktop__cell"}}) 
    price = soup.find_all(attrs={"class":{"ListingItemPriceNew__content-HAVf2"}})
    year = soup.find_all(attrs={"class":{"ListingItem__year"}})
    kmage = soup.find_all(attrs={"class":{"ListingItem__kmAge"}})
    model = soup.find_all(attrs={"class":{"ListingItem__summary"}})
    
    i_hate_sales = 0
    
    for j in np.arange(len(year)):
    
        otherparams = re.sub('\s+','', other[j*5].text).split('/') # тут сразу много инфы через '/'.
        
        if price[j+i_hate_sales].text[0:11] == 'со скидками': # трудно отделить цену со скидками от обычной,
            i_hate_sales +=1                                  #  у них одинаковый класс и тэг, но так работает.
        
        data = data.append({'Модель': model[j].a.text,
                             'Цена': re.sub('\W','', price[j+i_hate_sales].text),
                             'Год выпуска': year[j].text,
                             'Пробег': re.sub('\D','', kmage[j].text),
                             'Привод': other[j*5+3].text,
                             'ЛС': re.sub('\D','', otherparams[1]),
                             'Тип двигателя': re.sub('\s','', otherparams[2]),
                             'Тип кузова': other[j*5+2].text,
                             'Коробка передач': other[j*5+1].text,
                             'Объем двигателя': otherparams[0][:3],
                             'Цвет': other[j*5+4].text}, ignore_index=True)
        
    share = browser.find_element(By.XPATH, "//*[contains(text(), 'Следующая, Cmd ')]")
    share.click() # Нажимаем на кнопку 'следующая страница'.
    time.sleep(5) # Если слишком часто отправлять запросы, серверу не понравится.

In [35]:
# Так как у электромобилей нет объема двигателя, все немного рушится. Там, где у всех мощность в л.c.,
# у электро — КВт, где объем двигателя, там — мощность в л.c. Сейчас мы это исправим!

data.loc[data['Тип двигателя'] == 'Электро', 'ЛС'] = data.loc[data['Тип двигателя'] == 'Электро']['Объем двигателя']
data.loc[data['Тип двигателя'] == 'Электро', 'Объем двигателя'] = np.NaN
data.loc[data['Тип двигателя'] == 'Электро', 'Запас хода'] = data.loc[data['Тип двигателя'] == 'Электро']['Коробка передач']
data.loc[data['Тип двигателя'] == 'Электро', 'Коробка передач'] = np.NaN

# Готово. Есть еще жесткая Тесла, у нее 1020 сил. Кто ж знал.
data.loc[data['Модель'] == 'Tesla Model X Plaid I Рестайлинг', 'ЛС'] = 1020

# Ну вот и все! Парсили 12.05.2023 
data.to_csv('parsing.csv')